In [24]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn import tree

In [35]:
train = pd.read_csv("../data/train.csv", delimiter=';')

In [36]:
train.head()

,target,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q1,r4h1,r4h2,...,computer,television,qmobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,rural,age
0,0,0.0,0,4,0,1,1,0,0,1,...,0,0,3,1,0,0,0,0,1,43
1,0,0.0,0,8,0,1,1,1,0,1,...,0,0,3,0,0,0,0,0,1,18
2,0,0.0,0,5,0,1,1,0,0,2,...,0,1,2,0,0,1,0,0,0,62
3,1,0.0,0,8,0,1,1,2,0,2,...,1,1,4,1,0,0,0,0,0,20
4,0,350000.0,0,5,0,1,1,3,1,1,...,1,0,3,1,0,0,0,0,0,3


In [37]:
train["overcrowding"] = train["overcrowding"].str.replace(",", ".").astype("float")
train["meaneduc"] = train["meaneduc"].str.replace(",", ".").astype("float")

In [38]:
test = train.sample(frac=0.2)
train = train.drop(test.index)

In [39]:
X = train.drop(columns="target")
Y = train["target"]

In [40]:
clf = RandomForestClassifier(n_estimators=100, max_depth=40)
clf = clf.fit(X, Y)

In [41]:
clf.score(test.drop(columns="target"), test["target"])

0.9286649214659686